In [1]:
%load_ext autoreload
%autoreload 2
from utils import dataloader, utils, calibrate_imu_stereo
import torch
from models.resnet34_encoder_decoder import ResNet, FCN8s
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.transform import Rotation
import numpy as np
import math
from tqdm import tqdm
import gc
import torchvision.models as models
import time

### Set seed for reproducibility

In [2]:
np.random.seed(1133)

### returns imu and stereo rotation matrices for a sequence

In [3]:
def rots_per_sequence(dataset, model, norm, calibration_mat):
    """
    Computes roll and pitch via stereo and aligns it to the imu. Performs it on the whole sequence.
    
    Parameters
    ----------
    dataset : torch dataset
        contains stereo images, calibration data and imu data
    model : nn.Model
        segmentation CNN (VGG based Encoder Decoder)
    norm : torch transformation
        function to normalize images for CNN input
    calibration_mat : np.array
        calibration matrix to align stereo and imu
    return: tuple
        tuple[0] : stereo estimated orientation as 3x3 matrix (already calibrated to imu)
        tuple[1] : imu orientation as 3x3 matrix
        tuple[2] : inference times
        tuple[3] : disparity and dept computation time
    """

    stereo_calibrated_mats = []
    stereo_uncalibrated_mats = []
    imu_mats = []
    prediction_time = []
    pc_computation_time = []

    for j in tqdm(range(len(dataset))):
        sample = dataset[j]
        
        img_l = sample["img_l"] / 255
        K_l = sample["K_l"]
        D_l = sample["D_l"]
        R_l = sample["R_l"]
        P_l = sample["P_l"]
        img_r = sample["img_r"] / 255
        K_r = sample["K_r"]
        D_r = sample["D_r"]
        R_r = sample["R_r"]
        P_r = sample["P_r"]
        R = sample["R"]
        T = sample["T"]
        imu = sample["imu"]
    
        # Intrisic rotation aerospace sequence (Z-Y’-X”) yaw, pitch and roll
        # quaternion to matrix conversion: https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToMatrix/index.htm
        imu = Rotation.from_quat(imu).as_euler("ZYX", degrees=False)
        imu_mat = Rotation.from_euler("ZYX", [0., imu[1], imu[2]], degrees=False).as_matrix()

        map1_left, map2_left = cv2.initUndistortRectifyMap(K_l, D_l, R_l, P_l, (img_l.shape[1], img_l.shape[0]), cv2.CV_32FC1)
        map1_right, map2_right = cv2.initUndistortRectifyMap(K_r, D_r, R_r, P_r, (img_r.shape[1], img_r.shape[0]), cv2.CV_32FC1)
        
        img_l = cv2.remap(img_l, map1_left, map2_left, cv2.INTER_LINEAR)
        img_r = cv2.remap(img_r, map1_right, map2_right, cv2.INTER_LINEAR)

        ################ estimate horizon with plane regression ####################################################################
        res_img = cv2.resize(img_l, dsize=(960, 640), interpolation=cv2.INTER_CUBIC)
        # Perform forward pass with resized image as input tensor
        trans1 = transforms.ToTensor()
        input_tensor = norm(trans1(res_img)).to("cuda:0")
        prediction_begin = time.time()
        with torch.no_grad():
            output = model(input_tensor.unsqueeze(0).float())
        prediction_end = time.time()
        prediction_time.append(prediction_end-prediction_begin)
        # Binarize output with a high threshold (less false positives (but also less true positives))
        output = utils.binarize(torch.squeeze(output.data.cpu()), activation="sigmoid", threshold=0.5).numpy()
        res_out = cv2.resize(output, dsize=(img_l.shape[1], img_l.shape[0]), interpolation=cv2.INTER_CUBIC)
        res_out *= 255
        res_out = res_out.astype(np.uint8)
        # # Mask original image
        img_l_masked = cv2.bitwise_and(img_l, img_l, mask = res_out)

        img_l_masked_resized, img_r_resized, P_l_resize, P_r_resize = utils.resize_img_and_projection(img_l_masked, img_r, P_l, P_r, 0.5)

        pc_begin = time.time()
        pc_water, disparity = utils.compute_pointcloud((img_l_masked_resized * 255).astype("uint8"), (img_r_resized * 255).astype("uint8"), 
                                                        P_l_resize, P_r_resize, minDisparity=0, numDisparities=160, blocksize=9)
        pc_end = time.time()
        pc_computation_time.append(pc_end-pc_begin)
        #######################################
        pc_water = pc_water[pc_water[:,2]<=40]

        plane_begin = time.time()
        normal, support = utils.calc_plane(pc_water, var_index=[0,2,1])
        normal = np.asarray(normal)
        support = np.asarray(support)
        normal = 1 / np.sqrt(np.sum(normal**2)) * normal
        plane_end = time.time()
        #print("plane ", plane_end-plane_begin)
        
        roll, pitch = utils.normal_to_euler(normal, degrees=False)
        stereo_uncalibrated_mat = Rotation.from_euler("ZYX", [0., pitch, roll], degrees=False).as_matrix()
        end = time.time()
        #print("time", end-start)
        stereo_calibrated_mat = np.dot(stereo_uncalibrated_mat, calibration_mat)
        stereo_uncalibrated_mats.append(stereo_uncalibrated_mat)
        stereo_calibrated_mats.append(stereo_calibrated_mat)
        imu_mats.append(imu_mat)
                
    return stereo_calibrated_mats, imu_mats, prediction_time, pc_computation_time

### average misorientation angle (https://jsdajournal.springeropen.com/articles/10.1186/s40488-015-0032-x)

In [4]:
def print_ama(imu_mats, stereo_mats):
    """
    Computes and prints the AMA and confidence intervals
    
    Parameters
    ----------
    imu_mats : list
        list of imu measurements per timestep (3x3 matrices) 
    stereo_mats : list
        list of stereo estimated orientation per timestep (3x3 matrices)
    """
    ### create dataset for bootstrapping
    diff_mats = []
    for i in range(len(imu_mats)):
        stereo_calibrated_mat = stereo_mats[i]
        imu_mat = imu_mats[i]
        diff_mats.append(np.dot(imu_mat, stereo_calibrated_mat.T))
    diff_mats = np.array(diff_mats)

    ### compute average misorientation angle
    sum_diff_mats = np.zeros((3,3))
    for i in range(diff_mats.shape[0]):
        sum_diff_mats += diff_mats[i]
    avg_diff_mat = sum_diff_mats / diff_mats.shape[0]
    U,S,W = np.linalg.svd(avg_diff_mat)
    M = np.dot(U,W)

    AMA = []
    for i in range(diff_mats.shape[0]):
        mis = np.arccos((np.trace(np.dot(diff_mats[i].T, M)) -1) / 2)
        AMA.append(mis)
    print("Misorientation angle in degree: ", np.mean(np.degrees(AMA)))


    ### Bootstrapping ###
    bootstrepping_iterations = 1000
    AMA_per_sample = []
    for k in range(bootstrepping_iterations):
        sample_idx = np.random.randint(0, diff_mats.shape[0], diff_mats.shape[0])
        sample = diff_mats[sample_idx]

        sum_diff_mats = np.zeros((3,3))
        for i in range(sample.shape[0]):
            sum_diff_mats += sample[i]
        avg_diff_mat = sum_diff_mats / sample.shape[0]
        U,S,W = np.linalg.svd(avg_diff_mat)
        M = np.dot(U,W)

        AMA = []
        for i in range(sample.shape[0]):
            mis = np.arccos((np.trace(np.dot(sample[i].T, M)) -1) / 2)
            AMA.append(mis)

        sum_ama = 0
        for i in range(len(AMA)):
            sum_ama += AMA[i]
        AMA_per_sample.append(np.degrees(sum_ama / len(AMA)))

    print("Confidence Intervalls in degrees (2.5, 97.5) quantile: ", np.quantile(AMA_per_sample, 0.025), np.quantile(AMA_per_sample, 0.975))

In [5]:
def get_gyro_measurments(dataset):
    """
    return gyro measurements and integrated orientations
    
    Parameters
    ----------
    dataset : torch dataset
        contains gyro measurements 
    
    return: tuple
        tuple[0] : integrated orientations 
        tuple[1] : gyro measurements
    """
    integrated_mats = []
    gyro_mats = [Rotation.from_quat(dataset[0]["dq"]).as_matrix()]
    integrated_measurements = Rotation.from_quat(dataset[0]["imu"]).as_matrix()
    integrated_mats.append(integrated_measurements)
    for i in range(1, len(dataset)): 
        gyro = dataset[i]["dq"]
        gyro = Rotation.from_quat(gyro).as_matrix()
        integrated_measurements = integrated_measurements @ gyro
        integrated_mats.append(integrated_measurements)
        gyro_mats.append(gyro)
    return integrated_mats, gyro_mats

In [6]:
def complementary_filter(stereo_mats, gyro_mats):
    """
    Fusion of stereo estimation and gyro data via complementary filter
    
    Parameters
    ----------
    stereo_mats : list
        contains stereo orientation measurements (3x3 matrices)
    gyro_mats : list
        contains gyro measurements (3x3 matrices)
    
    return: list
        fused orientation (3x3 matrices)
    """
    roll = 0
    pitch = 0
    filtered_mats = []
    for i in range(len(stereo_mats)):
        stereo_euler = Rotation.from_matrix(stereo_mats[i]).as_euler("ZYX", degrees=False)
        gyro_euler = Rotation.from_matrix(gyro_mats[i]).as_euler("ZYX", degrees=False)
        stereo_pitch = stereo_euler[1]
        stereo_roll = stereo_euler[2]
        gyro_pitch = gyro_euler[1]
        gyro_roll = gyro_euler[2]

        # filter
        roll = 0.8 * (roll + (gyro_roll * (1. / 1))) + 0.2 * stereo_roll        
        pitch = 0.8 * (pitch + (gyro_pitch * (1. / 1))) + 0.2 * stereo_pitch

        filtered_mats.append(Rotation.from_euler("ZYX", [0, pitch, roll]).as_matrix())
    return filtered_mats

### load model parameters

In [7]:
RESNET_MODEL_PATH = "models/ResNet34_Enc_Dec_Misc_Constance"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet34 = models.resnet34(pretrained=True)
res_model = ResNet(resnet34)
model = FCN8s(res_model, 1).to(device)
model.load_state_dict(torch.load(RESNET_MODEL_PATH))
model.eval()
# Mean and standard deviation for all channels of the 'WaterMisc' Dataset
mean, std = ([0.4454203248023987, 0.4749860167503357, 0.4680652916431427], [0.2575828433036804, 0.2523757517337799, 0.2858140468597412])

# Transformation to normalize and unnormalize input images
norm = transforms.Normalize(mean, std)

#### Calibration Rotation for day_1

In [8]:
plt.rcParams['figure.figsize'] = [8, 8]
calibration_dataset = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="calib")
imu_stereo_calib = calibrate_imu_stereo.ImuStereoCalibration(model, norm, calibration_dataset, seq_from = 0, seq_to = len(calibration_dataset), degrees=True, debug=False, resize_factor=0.5)
calibration_mat_day_1 = imu_stereo_calib.calibrate(minDisparity = 0, numDisparities=160, blocksize=9)
del calibration_dataset

100%|██████████| 596/596 [06:46<00:00,  1.46it/s]


#### Evaluation day 1, sequence 0

In [9]:
dataset_d1_0 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="0")
stereo_mats_0, imu_mats_0, prediction_time_0, pc_computation_time_0 =  rots_per_sequence(dataset_d1_0, model, norm, calibration_mat_day_1)
integrated_gyro_mats_0, gyro_mats_0 = get_gyro_measurments(dataset_d1_0)
filtered_mats_0 = complementary_filter(stereo_mats_0, gyro_mats_0)
print_ama(imu_mats_0, stereo_mats_0)
print_ama(imu_mats_0, integrated_gyro_mats_0)
print_ama(imu_mats_0, filtered_mats_0)

100%|██████████| 753/753 [08:23<00:00,  1.50it/s]


Misorientation angle in degree:  0.13842858238313802
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.13318808628002246 0.14324071130550936
Misorientation angle in degree:  11.672481132405382
Confidence Intervalls in degrees (2.5, 97.5) quantile:  11.35407607486328 11.946847155682205
Misorientation angle in degree:  0.12901438273315188
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12229580355291544 0.136683255318897


In [10]:
del dataset_d1_0

#### Evaluation day 1, sequence 1

In [11]:
dataset_d1_1 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="1")
stereo_mats_1, imu_mats_1, prediction_time_1, pc_computation_time_1 = rots_per_sequence(dataset_d1_1, model, norm, calibration_mat_day_1)
integrated_gyro_mats_1, gyro_mats_1 = get_gyro_measurments(dataset_d1_1)
filtered_mats_1 = complementary_filter(stereo_mats_1, gyro_mats_1)
print_ama(imu_mats_1, stereo_mats_1)
print_ama(imu_mats_1, integrated_gyro_mats_1)
print_ama(imu_mats_1, filtered_mats_1)

100%|██████████| 1119/1119 [12:36<00:00,  1.48it/s]


Misorientation angle in degree:  0.19719432711369445
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.18776469366665804 0.20612046563500974
Misorientation angle in degree:  26.347055061096164
Confidence Intervalls in degrees (2.5, 97.5) quantile:  24.715412266068892 27.984835271347404
Misorientation angle in degree:  0.19231671477011017
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.18101399411559596 0.2042497134698251


In [12]:
del dataset_d1_1

#### Evaluation day 1, sequence 2

In [13]:
dataset_d1_2 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="2")
stereo_mats_2, imu_mats_2, prediction_time_2, pc_computation_time_2 = rots_per_sequence(dataset_d1_2, model, norm, calibration_mat_day_1)
integrated_gyro_mats_2, gyro_mats_2 = get_gyro_measurments(dataset_d1_2)
filtered_mats_2 = complementary_filter(stereo_mats_2, gyro_mats_2)
print_ama(imu_mats_2, stereo_mats_2)
print_ama(imu_mats_2, integrated_gyro_mats_2)
print_ama(imu_mats_2, filtered_mats_2)

100%|██████████| 826/826 [08:54<00:00,  1.54it/s]


Misorientation angle in degree:  0.16498398707581174
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.15800558203454668 0.1721152417509462
Misorientation angle in degree:  8.646174361316849
Confidence Intervalls in degrees (2.5, 97.5) quantile:  8.362556967521389 8.8894720054816
Misorientation angle in degree:  0.1288491061978574
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12189627847994279 0.13683006547753424


In [14]:
del dataset_d1_2

#### Evaluation day 1, sequence 3

In [15]:
dataset_d1_3 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="3")
stereo_mats_3, imu_mats_3, prediction_time_3, pc_computation_time_3 = rots_per_sequence(dataset_d1_3, model, norm, calibration_mat_day_1)
integrated_gyro_mats_3, gyro_mats_3 = get_gyro_measurments(dataset_d1_3)
filtered_mats_3 = complementary_filter(stereo_mats_3, gyro_mats_3)
print_ama(imu_mats_3, stereo_mats_3)
print_ama(imu_mats_3, integrated_gyro_mats_3)
print_ama(imu_mats_3, filtered_mats_3)

100%|██████████| 2390/2390 [26:55<00:00,  1.48it/s]


Misorientation angle in degree:  0.1796795228352441
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.17480055493402694 0.1844152015839702
Misorientation angle in degree:  23.864626083618727
Confidence Intervalls in degrees (2.5, 97.5) quantile:  23.367326493563134 24.34821713591673
Misorientation angle in degree:  0.1282025135776476
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.12177351412433142 0.1354882707883505


In [16]:
del dataset_d1_3

#### Evaluation day 1, sequence 4

In [17]:
dataset_d1_4 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=1, sequence="4")
stereo_mats_4, imu_mats_4, prediction_time_4, pc_computation_time_4 = rots_per_sequence(dataset_d1_4, model, norm, calibration_mat_day_1)
integrated_gyro_mats_4, gyro_mats_4 = get_gyro_measurments(dataset_d1_4)
filtered_mats_4 = complementary_filter(stereo_mats_4, gyro_mats_4)
print_ama(imu_mats_4, stereo_mats_4)
print_ama(imu_mats_4, integrated_gyro_mats_4)
print_ama(imu_mats_4, filtered_mats_4)

100%|██████████| 597/597 [07:11<00:00,  1.38it/s]


Misorientation angle in degree:  0.15520477824184073
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1464076823135768 0.16270641008457057
Misorientation angle in degree:  8.180498987118323
Confidence Intervalls in degrees (2.5, 97.5) quantile:  7.713077945132384 8.585811252123278
Misorientation angle in degree:  0.1410983772735712
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.13186285136420992 0.15303439435094235


In [18]:
del dataset_d1_4

#### Calibration Rotation for day_2

In [19]:
plt.rcParams['figure.figsize'] = [8, 8]
calibration_dataset = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="calib")
imu_stereo_calib = calibrate_imu_stereo.ImuStereoCalibration(model, norm, calibration_dataset, seq_from = 0, seq_to = len(calibration_dataset), degrees=True, debug=False, resize_factor=0.5)
calibration_mat_day_2 = imu_stereo_calib.calibrate(minDisparity = 0, numDisparities=160, blocksize=9)
del calibration_dataset

100%|██████████| 596/596 [06:49<00:00,  1.46it/s]


#### Evaluation day 2, sequence 5

In [20]:
dataset_d2_5 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="5")
stereo_mats_5, imu_mats_5, prediction_time_5, pc_computation_time_5 =  rots_per_sequence(dataset_d2_5, model, norm, calibration_mat_day_2)
integrated_gyro_mats_5, gyro_mats_5 = get_gyro_measurments(dataset_d2_5)
filtered_mats_5 = complementary_filter(stereo_mats_5, gyro_mats_5)
print_ama(imu_mats_5, stereo_mats_5)
print_ama(imu_mats_5, integrated_gyro_mats_5)
print_ama(imu_mats_5, filtered_mats_5)

100%|██████████| 597/597 [06:42<00:00,  1.48it/s]


Misorientation angle in degree:  0.18873418873622252
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.17893019915658173 0.1984988730946124
Misorientation angle in degree:  35.802442798489615
Confidence Intervalls in degrees (2.5, 97.5) quantile:  33.52071105412884 37.80321979104047
Misorientation angle in degree:  0.14539040977462522
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.13511466570572417 0.1572005621263549


In [21]:
del dataset_d2_5

#### Evaluation day 2, sequence 6

In [22]:
dataset_d2_6 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="6")
stereo_mats_6, imu_mats_6, prediction_time_6, pc_computation_time_6 = rots_per_sequence(dataset_d2_6, model, norm, calibration_mat_day_2)
integrated_gyro_mats_6, gyro_mats_6 = get_gyro_measurments(dataset_d2_6)
filtered_mats_6 = complementary_filter(stereo_mats_6, gyro_mats_6)
print_ama(imu_mats_6, stereo_mats_6)
print_ama(imu_mats_6, integrated_gyro_mats_6)
print_ama(imu_mats_6, filtered_mats_6)

100%|██████████| 439/439 [05:27<00:00,  1.34it/s]


Misorientation angle in degree:  0.19406901436583535
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1839584087990874 0.20340023704129973
Misorientation angle in degree:  5.717352926382113
Confidence Intervalls in degrees (2.5, 97.5) quantile:  5.498613672928683 5.910491812378123
Misorientation angle in degree:  0.16501527207126954
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1423209943436659 0.19279771082394134


In [23]:
del dataset_d2_6

#### Evaluation day 2, sequence 7

In [24]:
dataset_d2_7 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="7")
stereo_mats_7, imu_mats_7, prediction_time_7, pc_computation_time_7 = rots_per_sequence(dataset_d2_7, model, norm, calibration_mat_day_2)
integrated_gyro_mats_7, gyro_mats_7 = get_gyro_measurments(dataset_d2_7)
filtered_mats_7 = complementary_filter(stereo_mats_7, gyro_mats_7)
print_ama(imu_mats_7, stereo_mats_7)
print_ama(imu_mats_7, integrated_gyro_mats_7)
print_ama(imu_mats_7, filtered_mats_7)

100%|██████████| 1016/1016 [12:15<00:00,  1.38it/s]


Misorientation angle in degree:  0.6336928273913028
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.5872320087599691 0.6861549828623473
Misorientation angle in degree:  11.997248570189196
Confidence Intervalls in degrees (2.5, 97.5) quantile:  11.472385834814448 12.454951204792158
Misorientation angle in degree:  0.3387263225405888
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.3223681383592588 0.35606653255329723


In [25]:
del dataset_d2_7

#### Evaluation day 2, sequence 8

In [26]:
dataset_d2_8 = dataloader.CONSTANCEDATASET("/media/dennis/Extreme SSD/constance_stereo_imu", day=2, sequence="8")
stereo_mats_8, imu_mats_8, prediction_time_8, pc_computation_time_8 = rots_per_sequence(dataset_d2_8, model, norm, calibration_mat_day_2)
integrated_gyro_mats_8, gyro_mats_8 = get_gyro_measurments(dataset_d2_8)
filtered_mats_8 = complementary_filter(stereo_mats_8, gyro_mats_8)
print_ama(imu_mats_8, stereo_mats_8)
print_ama(imu_mats_8, integrated_gyro_mats_8)
print_ama(imu_mats_8, filtered_mats_8)

100%|██████████| 584/584 [06:36<00:00,  1.47it/s]


Misorientation angle in degree:  0.45863957097043534
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.43586642729154396 0.48363823121199223
Misorientation angle in degree:  13.018556755516462
Confidence Intervalls in degrees (2.5, 97.5) quantile:  12.358184998016586 13.621904818574931
Misorientation angle in degree:  0.2497418463619289
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.2346578684669472 0.2663957714534918


In [27]:
del dataset_d2_8

In [28]:
imu_mats = (imu_mats_0 + imu_mats_1)
stereo_mats = (stereo_mats_0 + stereo_mats_1 + stereo_mats_2 + stereo_mats_3 + stereo_mats_4 + stereo_mats_5 + stereo_mats_6 + stereo_mats_7 + stereo_mats_8)
integrated_gyro_mats = (integrated_gyro_mats_0 + integrated_gyro_mats_1 + integrated_gyro_mats_2 + integrated_gyro_mats_3 + integrated_gyro_mats_4 + integrated_gyro_mats_5 + integrated_gyro_mats_6 + integrated_gyro_mats_7 + integrated_gyro_mats_8)
filtered_mats = (filtered_mats_0 + filtered_mats_1 + filtered_mats_2 + filtered_mats_3 + filtered_mats_4 + filtered_mats_5 + filtered_mats_6 + filtered_mats_7 + filtered_mats_8)

print_ama(imu_mats, stereo_mats)
print_ama(imu_mats, integrated_gyro_mats)
print_ama(imu_mats, filtered_mats)

Misorientation angle in degree:  0.18047845549779967
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.1743223036829067 0.18646481716351135
Misorientation angle in degree:  40.6183701836332
Confidence Intervalls in degrees (2.5, 97.5) quantile:  39.47746239351728 41.71314227595228
Misorientation angle in degree:  0.17392680238724487
Confidence Intervalls in degrees (2.5, 97.5) quantile:  0.16634008852718826 0.18164514376044422


### Avg prediction time and avg disparity and reconstruction computation time

In [29]:
prediction_times = prediction_time_0 + prediction_time_1 + prediction_time_2 + prediction_time_3 + prediction_time_4 + prediction_time_5 + prediction_time_6 + prediction_time_7 + prediction_time_8
pc_computation_times = pc_computation_time_0 + pc_computation_time_1 + pc_computation_time_2 + pc_computation_time_3 + pc_computation_time_4 + pc_computation_time_5 + pc_computation_time_6 + pc_computation_time_7 + pc_computation_time_8

print("Avg prediction time: ", np.array(prediction_times).mean())
print("Avg pc computation time: ", np.array(pc_computation_times).mean())

Avg prediction time:  0.005100760232397977
Avg pc computation time:  0.2590683306829386
